# Gestor de Fotos y Videos - Organizador Inteligente

**Características principales:**
- Detección de duplicados por contenido (hash)
- Organización automática por año/mes
- Eliminación de archivos con sufijo "_2"
- Vista previa antes de ejecutar cambios
- Modo "prueba" para verificar sin modificar archivos

**IMPORTANTE:** Cada módulo es independiente

**Fuentes:** Cámara, Drone, Teléfono

## Importar Librerías (EJECUTAR PRIMERO)

Importa todas las librerías necesarias. Ejecuta esta celda antes de usar cualquier módulo.

In [ ]:
import os
import shutil
import hashlib
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import time
from pprint import pprint

print("Librerías importadas correctamente")
print(f"Fecha y hora actual: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Configuración Global (EJECUTAR SEGUNDO)

Define las rutas base que usarán los módulos

In [ ]:
# ========== CONFIGURACIÓN PRINCIPAL ==========

# Carpeta donde tienes fotos sin organizar
CARPETA_ORIGEN = Path(r"C:\Users\Alejandro\Documents\Fotos_Videos\108_FUJI")

# Carpeta destino en tu disco duro externo (SOLO la carpeta base, el código creará año/mes automáticamente)
CARPETA_DESTINO = Path(r"D:\Fotos")

# Carpeta para backups (opcional)
CARPETA_BACKUP = Path(r"C:\Users\Alejandro\Documents\Fotos_Videos\Backup")

# Extensiones válidas de archivos multimedia
VALID_EXTENSIONS = {
    '.jpg', '.jpeg', '.png', '.heic', '.heif', '.gif', '.bmp', '.tiff', '.raw',
    '.mp4', '.mov', '.avi', '.mkv', '.wmv', '.flv', '.m4v', '.mpg', '.mpeg'
}
VALID_EXTENSIONS = {ext.lower() for ext in VALID_EXTENSIONS}

# Meses en español
MESES_ESPAÑOL = {
    1: '01_Enero', 2: '02_Febrero', 3: '03_Marzo', 4: '04_Abril',
    5: '05_Mayo', 6: '06_Junio', 7: '07_Julio', 8: '08_Agosto',
    9: '09_Septiembre', 10: '10_Octubre', 11: '11_Noviembre', 12: '12_Diciembre'
}

print("Configuración cargada:")
print(f"  Origen: {CARPETA_ORIGEN}")
print(f"  Destino: {CARPETA_DESTINO}")
print(f"  Extensiones válidas: {len(VALID_EXTENSIONS)} tipos")

# Verificar rutas
if CARPETA_ORIGEN.exists():
    print(f"  Carpeta origen: OK")
else:
    print(f"  ADVERTENCIA: Carpeta origen NO existe")

if CARPETA_DESTINO.exists():
    print(f"  Carpeta destino: OK")
else:
    print(f"  ADVERTENCIA: Carpeta destino NO existe")

## MÓDULO 1: Escanear Archivos

Escanea una carpeta y encuentra todos los archivos multimedia.
Ejecuta este módulo de forma independiente para ver qué archivos tienes.

In [ ]:
def escanear_archivos(carpeta_a_escanear, recursivo=False):
    """
    Escanea y retorna todos los archivos multimedia.
    
    Args:
        carpeta_a_escanear: Path de la carpeta a escanear
        recursivo: True = escanea subcarpetas, False = solo la carpeta indicada
    
    Uso:
        archivos = escanear_archivos(CARPETA_ORIGEN, recursivo=False)
    """
    archivos_encontrados = []
    
    if not carpeta_a_escanear.exists():
        print(f"ERROR: La carpeta no existe: {carpeta_a_escanear}")
        return archivos_encontrados
    
    print(f"Escaneando: {carpeta_a_escanear}")
    print(f"Modo: {'Recursivo (incluye subcarpetas)' if recursivo else 'Solo esta carpeta'}")
    
    if recursivo:
        # Escanear recursivamente todas las subcarpetas
        for archivo in carpeta_a_escanear.rglob("*"):
            if archivo.is_file() and archivo.suffix.lower() in VALID_EXTENSIONS:
                archivos_encontrados.append(archivo)
    else:
        # Escanear solo archivos en esta carpeta (no subcarpetas)
        for archivo in carpeta_a_escanear.glob("*"):
            if archivo.is_file() and archivo.suffix.lower() in VALID_EXTENSIONS:
                archivos_encontrados.append(archivo)
    
    return archivos_encontrados


# ========== EJECUTAR ESCANEO ==========
# Cambia la ruta si quieres escanear otra carpeta
CARPETA_A_ESCANEAR = CARPETA_ORIGEN

# Configurar si quieres escanear subcarpetas o no
ESCANEO_RECURSIVO = False  # False = solo la carpeta indicada, True = incluye subcarpetas

archivos_encontrados = escanear_archivos(CARPETA_A_ESCANEAR, recursivo=ESCANEO_RECURSIVO)

print(f"\nResultados del escaneo:")
print(f"Total de archivos multimedia: {len(archivos_encontrados)}")

# Estadísticas por tipo
stats_tipo = defaultdict(int)
for archivo in archivos_encontrados:
    stats_tipo[archivo.suffix.lower()] += 1

print(f"\nDesglose por tipo:")
for ext, count in sorted(stats_tipo.items()):
    print(f"  {ext}: {count} archivo(s)")

# Mostrar ejemplos
if archivos_encontrados:
    print(f"\nPrimeros 10 archivos:")
    for archivo in archivos_encontrados[:10]:
        tamaño_mb = archivo.stat().st_size / (1024 * 1024)
        print(f"  - {archivo.name} ({tamaño_mb:.2f} MB)")

## MÓDULO 2: Detectar Duplicados por Hash

Detecta archivos duplicados comparando su contenido (no solo el nombre).
Este módulo es independiente. Usa 'archivos_encontrados' del Módulo 1.

In [ ]:
def calcular_hash(archivo_path):
    """Calcula el hash MD5 de un archivo."""
    hash_obj = hashlib.md5()
    try:
        with open(archivo_path, 'rb') as f:
            for bloque in iter(lambda: f.read(4096), b''):
                hash_obj.update(bloque)
        return hash_obj.hexdigest()
    except Exception as e:
        print(f"Error calculando hash de {archivo_path.name}: {e}")
        return None


def encontrar_duplicados(lista_archivos, mostrar_progreso=True):
    """
    Encuentra duplicados por contenido.
    Retorna dict {hash: [lista de archivos duplicados]}
    """
    hashes = defaultdict(list)
    total = len(lista_archivos)
    
    print(f"Calculando hashes de {total} archivos...")
    
    for i, archivo in enumerate(lista_archivos, 1):
        if mostrar_progreso and i % 50 == 0:
            print(f"  Progreso: {i}/{total} ({i*100//total}%)")
        
        hash_val = calcular_hash(archivo)
        if hash_val:
            hashes[hash_val].append(archivo)
    
    # Filtrar solo duplicados
    duplicados = {h: archivos for h, archivos in hashes.items() if len(archivos) > 1}
    return duplicados


def eliminar_duplicados(duplicados_dict, dry_run=True, mantener='primero'):
    """
    Elimina archivos duplicados.
    
    Args:
        duplicados_dict: Diccionario de duplicados
        dry_run: True = solo muestra, False = elimina realmente
        mantener: 'primero' o 'ultimo' (cuál archivo mantener)
    """
    total_eliminados = 0
    total_errores = 0
    espacio_liberado = 0
    
    modo = "SIMULACIÓN" if dry_run else "ELIMINACIÓN REAL"
    print(f"\n{modo} de duplicados:")
    print(f"Criterio: Mantener el {mantener} archivo de cada grupo\n")
    
    for hash_val, lista_archivos in duplicados_dict.items():
        # Ordenar por fecha de modificación
        lista_ordenada = sorted(lista_archivos, key=lambda x: x.stat().st_mtime)
        
        # Decidir cuál mantener
        if mantener == 'primero':
            mantener_archivo = lista_ordenada[0]
            eliminar_archivos = lista_ordenada[1:]
        else:
            mantener_archivo = lista_ordenada[-1]
            eliminar_archivos = lista_ordenada[:-1]
        
        print(f"Grupo de {len(lista_archivos)} duplicados:")
        print(f"  Mantener: {mantener_archivo.name}")
        
        for archivo in eliminar_archivos:
            tamaño = archivo.stat().st_size
            espacio_liberado += tamaño
            
            if dry_run:
                print(f"  [SIMULAR] Eliminar: {archivo.name} ({tamaño/(1024*1024):.2f} MB)")
                total_eliminados += 1
            else:
                try:
                    archivo.unlink()
                    print(f"  Eliminado: {archivo.name} ({tamaño/(1024*1024):.2f} MB)")
                    total_eliminados += 1
                except Exception as e:
                    print(f"  ERROR eliminando {archivo.name}: {e}")
                    total_errores += 1
        print()
    
    print(f"\nResumen:")
    print(f"  Archivos eliminados: {total_eliminados}")
    print(f"  Errores: {total_errores}")
    print(f"  Espacio liberado: {espacio_liberado/(1024*1024):.2f} MB")
    
    return {
        'eliminados': total_eliminados,
        'errores': total_errores,
        'espacio_mb': espacio_liberado/(1024*1024)
    }


# ========== EJECUTAR DETECCIÓN DE DUPLICADOS ==========
# Configuración
DRY_RUN = True  # Cambiar a False para eliminar realmente
MANTENER = 'primero'  # 'primero' o 'ultimo'

# Si no tienes 'archivos_encontrados', ejecuta primero el Módulo 1
if 'archivos_encontrados' not in locals():
    print("ADVERTENCIA: Ejecuta primero el Módulo 1 (Escanear Archivos)")
    archivos_encontrados = escanear_archivos(CARPETA_ORIGEN)

if archivos_encontrados:
    duplicados = encontrar_duplicados(archivos_encontrados)
    
    print(f"\nResultados:")
    print(f"Grupos de duplicados encontrados: {len(duplicados)}")
    
    if duplicados:
        total_dup = sum(len(lista) - 1 for lista in duplicados.values())
        espacio_total = 0
        for lista in duplicados.values():
            for archivo in lista[1:]:
                espacio_total += archivo.stat().st_size
        
        print(f"Archivos duplicados a eliminar: {total_dup}")
        print(f"Espacio a liberar: {espacio_total/(1024*1024):.2f} MB")
        
        # Mostrar algunos ejemplos
        print(f"\nPrimeros 3 grupos de duplicados:")
        for i, (hash_val, lista) in enumerate(list(duplicados.items())[:3], 1):
            print(f"\nGrupo {i} ({len(lista)} copias):")
            for arch in lista:
                print(f"  - {arch.name}")
        
        # ELIMINAR DUPLICADOS
        print("\n" + "="*70)
        resultado = eliminar_duplicados(duplicados, dry_run=DRY_RUN, mantener=MANTENER)
        print("="*70)
    else:
        print("No se encontraron duplicados")
else:
    print("No hay archivos para procesar")

## MÓDULO 3: Eliminar Archivos con Sufijo "_2"

Encuentra y elimina archivos que terminan con "_2" (duplicados típicos al copiar).
Este módulo es completamente independiente.

In [ ]:
def encontrar_sufijo_2(lista_archivos):
    """Encuentra archivos con sufijo '_2' en el nombre."""
    return [archivo for archivo in lista_archivos if archivo.stem.endswith('_2')]


def eliminar_sufijo_2(archivos_2, dry_run=True, crear_backup=False):
    """
    Elimina archivos con sufijo _2.
    
    Args:
        archivos_2: Lista de archivos a eliminar
        dry_run: True = solo muestra, False = elimina
        crear_backup: True = crea backup antes de eliminar
    """
    total_eliminados = 0
    total_errores = 0
    espacio_liberado = 0
    
    modo = "SIMULACIÓN" if dry_run else "ELIMINACIÓN REAL"
    print(f"\n{modo} de archivos con sufijo '_2':")
    print(f"Total de archivos: {len(archivos_2)}\n")
    
    for archivo in archivos_2:
        tamaño = archivo.stat().st_size
        
        if crear_backup and not dry_run:
            CARPETA_BACKUP.mkdir(parents=True, exist_ok=True)
            backup_path = CARPETA_BACKUP / archivo.name
            try:
                shutil.copy2(str(archivo), str(backup_path))
                print(f"  Backup creado: {archivo.name}")
            except Exception as e:
                print(f"  ERROR creando backup de {archivo.name}: {e}")
        
        if dry_run:
            print(f"  [SIMULAR] Eliminar: {archivo.name} ({tamaño/(1024*1024):.2f} MB)")
            total_eliminados += 1
            espacio_liberado += tamaño
        else:
            try:
                archivo.unlink()
                print(f"  Eliminado: {archivo.name} ({tamaño/(1024*1024):.2f} MB)")
                total_eliminados += 1
                espacio_liberado += tamaño
            except Exception as e:
                print(f"  ERROR eliminando {archivo.name}: {e}")
                total_errores += 1
    
    print(f"\nResumen:")
    print(f"  Archivos eliminados: {total_eliminados}")
    print(f"  Errores: {total_errores}")
    print(f"  Espacio liberado: {espacio_liberado/(1024*1024):.2f} MB")
    
    return {
        'eliminados': total_eliminados,
        'errores': total_errores,
        'espacio_mb': espacio_liberado/(1024*1024)
    }


# ========== EJECUTAR ELIMINACIÓN DE SUFIJO _2 ==========
# Configuración
DRY_RUN = True  # Cambiar a False para eliminar realmente
CREAR_BACKUP = False  # True para crear backup antes de eliminar

# Si no tienes 'archivos_encontrados', ejecuta primero el Módulo 1
if 'archivos_encontrados' not in locals():
    print("ADVERTENCIA: Ejecuta primero el Módulo 1 (Escanear Archivos)")
    archivos_encontrados = escanear_archivos(CARPETA_ORIGEN)

if archivos_encontrados:
    archivos_con_2 = encontrar_sufijo_2(archivos_encontrados)
    
    print(f"Archivos con sufijo '_2' encontrados: {len(archivos_con_2)}")
    
    if archivos_con_2:
        espacio = sum(f.stat().st_size for f in archivos_con_2) / (1024*1024)
        print(f"Espacio que ocupan: {espacio:.2f} MB")
        
        print(f"\nPrimeros 10 archivos:")
        for archivo in archivos_con_2[:10]:
            print(f"  - {archivo.name}")
        
        if len(archivos_con_2) > 10:
            print(f"  ... y {len(archivos_con_2) - 10} más")
        
        # ELIMINAR
        print("\n" + "="*70)
        resultado = eliminar_sufijo_2(archivos_con_2, dry_run=DRY_RUN, crear_backup=CREAR_BACKUP)
        print("="*70)
    else:
        print("No se encontraron archivos con sufijo '_2'")
else:
    print("No hay archivos para procesar")

## MÓDULO 4: Organizar Archivos por Año/Mes

Mueve archivos a carpetas organizadas por año y mes según su fecha.
Este módulo es completamente independiente.

In [ ]:
def obtener_fecha_archivo(archivo):
    """Obtiene la fecha de creación del archivo."""
    try:
        timestamp = archivo.stat().st_ctime
    except:
        timestamp = archivo.stat().st_mtime
    return datetime.fromtimestamp(timestamp)


def calcular_ruta_destino(archivo, carpeta_base):
    """Calcula la ruta destino según año/mes del archivo."""
    fecha = obtener_fecha_archivo(archivo)
    año = str(fecha.year)
    mes = MESES_ESPAÑOL[fecha.month]
    return carpeta_base / año / mes / archivo.name


def mover_archivos_por_fecha(lista_archivos, carpeta_destino, dry_run=True):
    """
    Mueve archivos a carpetas organizadas por año/mes.
    
    Args:
        lista_archivos: Lista de archivos a mover
        carpeta_destino: Carpeta base destino
        dry_run: True = solo muestra, False = mueve realmente
    """
    movidos = 0
    errores = 0
    
    # Agrupar por destino para mostrar resumen
    por_destino = defaultdict(list)
    
    for archivo in lista_archivos:
        destino = calcular_ruta_destino(archivo, carpeta_destino)
        carpeta_dest = f"{destino.parts[-3]}/{destino.parts[-2]}"
        por_destino[carpeta_dest].append((archivo, destino))
    
    modo = "SIMULACIÓN" if dry_run else "MOVIMIENTO REAL"
    print(f"\n{modo} de archivos por fecha:")
    print(f"Total de archivos: {len(lista_archivos)}\n")
    
    print("Resumen por carpeta destino:")
    for carpeta, archivos_dest in sorted(por_destino.items()):
        print(f"  {carpeta}: {len(archivos_dest)} archivo(s)")
    
    print(f"\nDetalle de movimientos:")
    
    for archivo in lista_archivos:
        destino = calcular_ruta_destino(archivo, carpeta_destino)
        
        if dry_run:
            print(f"  [SIMULAR] {archivo.name}")
            print(f"            -> {destino.parent.name}/{destino.name}")
            movidos += 1
        else:
            try:
                # Crear carpetas si no existen
                destino.parent.mkdir(parents=True, exist_ok=True)
                
                # Si existe, renombrar
                if destino.exists():
                    contador = 1
                    while destino.exists():
                        nuevo_nombre = f"{destino.stem}_{contador}{destino.suffix}"
                        destino = destino.parent / nuevo_nombre
                        contador += 1
                
                shutil.move(str(archivo), str(destino))
                print(f"  Movido: {archivo.name}")
                print(f"          -> {destino.parent.name}/{destino.name}")
                movidos += 1
            except Exception as e:
                print(f"  ERROR moviendo {archivo.name}: {e}")
                errores += 1
    
    print(f"\nResumen:")
    print(f"  Archivos movidos: {movidos}")
    print(f"  Errores: {errores}")
    
    return {'movidos': movidos, 'errores': errores}


# ========== EJECUTAR ORGANIZACIÓN POR FECHA ==========
# Configuración
DRY_RUN = False  # Cambiar a False para mover realmente, True para simular

# Si no tienes 'archivos_encontrados', ejecuta primero el Módulo 1
if 'archivos_encontrados' not in locals():
    print("ADVERTENCIA: Ejecuta primero el Módulo 1 (Escanear Archivos)")
    archivos_encontrados = escanear_archivos(CARPETA_ORIGEN)

if archivos_encontrados and CARPETA_DESTINO.exists():
    # Vista previa de algunos archivos
    total_archivos = len(archivos_encontrados)
    
    if total_archivos <= 10:
        # Si hay 10 o menos, mostrar todos
        print(f"Vista previa de organización ({total_archivos} archivos):")
        archivos_preview = archivos_encontrados
    else:
        # Mostrar primeros 5 y últimos 5
        print(f"Vista previa de organización (primeros 5 y últimos 5 de {total_archivos} archivos):")
        archivos_preview = archivos_encontrados[:5] + archivos_encontrados[-5:]
    
    for i, archivo in enumerate(archivos_preview):
        destino = calcular_ruta_destino(archivo, CARPETA_DESTINO)
        fecha = obtener_fecha_archivo(archivo)
        print(f"\n  {archivo.name}")
        print(f"    Fecha: {fecha.strftime('%Y-%m-%d')}")
        print(f"    Destino: {destino.parts[-3]}/{destino.parts[-2]}/{destino.name}")
        
        # Separador entre primeros 5 y últimos 5
        if i == 4 and total_archivos > 10:
            print(f"\n  ... ({total_archivos - 10} archivos más) ...")
    
    # MOVER ARCHIVOS
    print("\n" + "="*70)
    resultado = mover_archivos_por_fecha(
        archivos_encontrados,
        CARPETA_DESTINO,
        dry_run=DRY_RUN
    )
    print("="*70)
elif not CARPETA_DESTINO.exists():
    print(f"ERROR: La carpeta destino no existe: {CARPETA_DESTINO}")
else:
    print("No hay archivos para procesar")

## Instrucciones de Uso

Cómo usar cada módulo de forma independiente:

In [ ]:
"""
PASO 1: Siempre ejecuta primero las celdas 1, 2 y 3 (imports y configuración)

MÓDULO 1 - Escanear Archivos:
- Ejecuta la celda 4
- Te muestra qué archivos multimedia tienes en la carpeta

MÓDULO 2 - Detectar y Eliminar Duplicados:
- Ejecuta la celda 5
- Cambia DRY_RUN = False para eliminar realmente
- Elige MANTENER = 'primero' o 'ultimo'

MÓDULO 3 - Eliminar Archivos con "_2":
- Ejecuta la celda 6
- Cambia DRY_RUN = False para eliminar realmente
- Activa CREAR_BACKUP = True si quieres backup

MÓDULO 4 - Organizar por Año/Mes:
- Ejecuta la celda 7
- Cambia DRY_RUN = False para mover realmente
- Asegúrate que CARPETA_DESTINO existe

---

FLUJOS DE TRABAJO COMUNES:

Solo mover fotos a carpetas:
1. Ejecutar celdas 1, 2, 3 (configuración)
2. Ejecutar celda 7 (MÓDULO 4)

Solo buscar duplicados:
1. Ejecutar celdas 1, 2, 3 (configuración)
2. Ejecutar celda 5 (MÓDULO 2)

Solo eliminar archivos "_2":
1. Ejecutar celdas 1, 2, 3 (configuración)
2. Ejecutar celda 6 (MÓDULO 3)

---

IMPORTANTE:
- Siempre ejecuta primero en modo DRY_RUN = True (simulación)
- Verifica los resultados antes de ejecutar en modo real
- Considera crear backups de archivos importantes
"""